In [6]:
import os 
from PIL import Image, ImageDraw, ImageEnhance
import numpy as np
import random 

In [15]:
def darken_half(image, angle=None):
    # Convert image to RGB
    img = image.convert("RGB")
    width, height = img.size
    
    # Randomly select which half of the image to darken
    darken_left = random.choice([True, False])
    
    # Define a random line angle if not provided
    angle = angle if angle is not None else random.uniform(0, 360)
    center_x, center_y = width // 2, height // 2
    line_length = min(width, height) * 2  # Make the line extend beyond the image boundaries
    
    # Calculate the endpoints of the line based on the angle
    angle_rad = np.deg2rad(angle)
    start_x = center_x - int(np.cos(angle_rad) * line_length)
    start_y = center_y - int(np.sin(angle_rad) * line_length)
    end_x = center_x + int(np.cos(angle_rad) * line_length)
    end_y = center_y + int(np.sin(angle_rad) * line_length)
    
    # Randomize the darkness factor (between 0.2 and 0.7, you can adjust this range)
    darkness_factor = random.uniform(0.2, 0.7)

    # Darken the selected half of the image (left or right of the line)
    img_data = np.array(img)

    # Loop through all pixels and decide whether to darken based on side of the line
    for y in range(height):
        for x in range(width):
            # Calculate the position of the pixel relative to the line
            dx = end_x - start_x
            dy = end_y - start_y
            # Calculate the line equation
            line_position = (dy * (x - start_x) - dx * (y - start_y))  # This gives the position relative to the line
            if darken_left:
                # Darken pixels on the left side of the line
                if line_position > 0:
                    img_data[y, x] = (img_data[y, x] * darkness_factor).astype(np.uint8)
            else:
                # Darken pixels on the right side of the line
                if line_position < 0:
                    img_data[y, x] = (img_data[y, x] * darkness_factor).astype(np.uint8)

    # Convert back to an image
    img = Image.fromarray(img_data)

    return img


In [19]:
dir = "../segmentation_model/sim_data/markers_20250314-181037/darken_half" 
output_path = dir.replace("darken_half", "darken_half_twice") 
os.makedirs(output_path, exist_ok=True)
image_path_list = []
for file in os.listdir(dir):
    if file.endswith(".png"):
        image_path_list.append(os.path.join(dir, file)) 
        image = Image.open(os.path.join(dir, file)) 
        modified_image = darken_half(image)
        modified_image_filepath = os.path.join(output_path,file.replace(".png", "_darken_half.png")) 
        modified_image.save(modified_image_filepath)